# Capstone Project Data Science - 2nd part
The project is to determine if the neighborhood where I will move will be the same as the current neighborhood

## 1. PostalCode

### Import Libraries

In [2]:
from bs4 import BeautifulSoup as bs
import urllib.request as rqst
import pandas as pd
import numpy as np

###  Scrape the Wikipedia page

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = rqst.urlopen(url)
soup = bs(page, "lxml")

In [4]:
#print(soup.prettify())
tab = soup.find('table', class_='wikitable')

### Creating DataFrame

In [5]:
column_names = ['PostalCode','Borough', 'Neighborhood'] 
neighborhoods = pd.DataFrame(columns=column_names)

for row in tab.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        neighborhoods = neighborhoods.append({'PostalCode':cells[0].find(text=True).strip(),
                                              'Borough': cells[1].find(text=True).strip(),
                                              'Neighborhood': cells[2].find(text=True).strip()}, ignore_index=True)

neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Cleaning Dataset

In [6]:
neighborhoods=neighborhoods[neighborhoods['Borough']!='Not assigned']
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [7]:
neighborhoods.groupby(['Borough']).count()

,PostalCode,Neighborhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Scarborough,17,17
West Toronto,6,6


In [8]:
neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace("/", ",")
neighborhoods.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


In [9]:
neighborhoods[neighborhoods['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [10]:
neighborhoods.shape

(103, 3)

## 2. Lat Lng

In [11]:
latlng = pd.read_csv('http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv')
latlng = latlng.rename(columns={'Postal Code':'PostalCode'})

In [12]:
latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
neighborhoods_latlng = pd.merge(neighborhoods, latlng, on='PostalCode')

In [17]:
neighborhoods_latlng.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
